In [2]:
import pandas as pd


#address of the BV-RC metadata
file_path = "BVBRC_genome_amr.csv"

Reading the CSV file

In [3]:
# The unprocessed dataframe
main_df = pd.read_csv(file_path)

Exploring the number of experiments

In [4]:
num_rows = main_df.shape[0]

# Print the number of rows
print("Number of experiments:", num_rows)

Number of experiments: 303519


Exploring the umber of Uniqe genomes in the BV-BRC MTB metadata

In [5]:
total_num_unique_genomes = main_df["Genome ID"].nunique()

print("The total number of unique genome IDs in the initial BV-BRC metadata is: {}".format(total_num_unique_genomes))

The total number of unique genome IDs in the initial BV-BRC metadata is: 27606


In [6]:
# List of unique genomes
unique_values = main_df['Genome ID'].unique()

# Specify the file path for the stroing the list of unique genomes
file_path = 'Unique_Genome_IDs.txt'

# Write unique values to the text file
with open(file_path, 'w') as file:

    for i, value in enumerate(unique_values):

      # Adding vertical space to all the lines except the first
      # line so there wont be any empty lines at the end of text
      # file for downloading data

      if i!=0 :

        file.write('\n')

      file.write(str(value))

print(f"Unique Genome IDs have been written to {file_path}.")

Unique Genome IDs have been written to Unique_Genome_IDs.txt.


## Sending the output to => PatricBrowserTools



## Sort the dataframe based on genome ID
### Creating a new organized csv file

### The reason for sorting the CSV file according to the genome ID is to decrease the computational complexity of creating the final csv file from quadratic in term of number of rows, to linear.

In [7]:
df_sorted = main_df.sort_values(by=['Genome ID', 'Antibiotic'])

# Replace 'sorted_file.csv' with the desired output file name
output_file_path = 'sorted_file.csv'

#Adding NaN to the 'Laboratory Typing Method', 'Vendor' columns if not mentioned
df_sorted[['Laboratory Typing Method', 'Vendor']] = df_sorted[['Laboratory Typing Method', 'Vendor']].applymap(lambda x: str(x) if pd.notna(x) and x != '' else 'NaN')

# Save the sorted DataFrame to a CSV file
df_sorted.to_csv(output_file_path, index=False)

print(f"DataFrame sorted and saved to '{output_file_path}'.")


DataFrame sorted and saved to 'sorted_file.csv'.


---------

#Creating a new CSV file to be used for Machine leaning and testign MTB++ v1

In [8]:
# Storing all the genomes as a dictionary to organize who Metadata sheet and
# creating a sheet that eachrow is about a specific genome ID, and columns are
# the experiments wheather computational or Real lab tests (AST)

# Assuming df_sorted is the initial dataframe
genome_dict = {}

for index, row in df_sorted.iterrows():

    genome_id = row['Genome ID']

    # Creating pheotypes by concatination name of antibiotic and method and
    # vendor ( if method or vendor are empty, they will be considered as N/A
    # in the list of phenotypes)

    key = f"{row['Antibiotic']}_{row['Laboratory Typing Method']}_{row['Vendor']}"


    #Result of each experiment
    value = {row['Resistant Phenotype']} 

    if genome_id not in genome_dict:
        # Dari harkat mikoni tooye dataframe sort shode, 
        # age be genome jadid barkhord kardy, the elemente
        # jadid tooye dictionary misazi 
        genome_dict[genome_id] = {}

    genome_dict[genome_id][key] = value

Organizaing the dictionary as a dataframe to be saved for ML

In [9]:
genomes_list = list(genome_dict)

# estekhraje tamami phenotype ha momken tooye dataframe
phenotypes = set()

for _, inner_dict in genome_dict.items():
    # tooye tamame klid haayeh genome_dict migarde ta 
    # harchi phenotype yekta voojood dare ro peyda kone!  
    phenotypes.update(inner_dict.keys())

Final_df = pd.DataFrame(columns=['Genome ID'] + list(phenotypes))

# Add genome IDs to the "Genome ID" column
Final_df['Genome ID'] = genomes_list


In [11]:
# Migarde tooye tamame genome ID haye daroon dictionary genome ID ta betoone 
# list tamme phenotype haro (klid haye dictionary) peyda kone
for ID in genomes_list:

  # tamame phenotype haye marboot be yek genome tooye list gharar migireh 
  # ta chek beshe va S ya R barashoon taieen beshe
  list_of_phenotypes=list(genome_dict[ID].keys())

  for pheno in list_of_phenotypes:
    
    Final_df.loc[Final_df['Genome ID'] == ID, pheno] = next(iter(genome_dict[ID][pheno]))
    

Sorting the columns alphabetically to make the csv file more readable for human eye

In [15]:
first_column = Final_df.iloc[:, :1]

remaining_columns = Final_df.iloc[:, 1:].sort_index(axis=1)

# Concatenate the first column with the sorted remaining columns
df_sorted_sorted_columns = pd.concat([first_column, remaining_columns], axis=1)


Storing the final dataframe

In [16]:
df_sorted_sorted_columns.to_csv("ML_.csv",index=False)